Support Vector Machines (SVMs)
====

Before we get started, let's run the next cell to import some libraries we will use. Don't worry about the details of this.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_blobs
from sklearn.svm import SVC

We'll also define a helpful function called "plot_svc_decision_function" to visualize simple SVM classification models, copied from the reading. Again, **don't worry about the contents of this function for now** - we can treat it as a black box.

**NOTE:** This function will only work as expected when A) we only have 2 features, and B) we only have two classes.

In [ ]:
def plot_svc_decision_function(model, ax=None, plot_support=True):
    """Plot the decision function for a 2D SVC"""
    if ax is None:
        ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # create grid to evaluate model
    x = np.linspace(xlim[0], xlim[1], 30)
    y = np.linspace(ylim[0], ylim[1], 30)
    Y, X = np.meshgrid(y, x)
    xy = np.vstack([X.ravel(), Y.ravel()]).T
    P = model.decision_function(xy).reshape(X.shape)
    
    # plot decision boundary and margins
    ax.contour(X, Y, P, colors='k',
               levels=[-1, 0, 1], alpha=0.5,
               linestyles=['--', '-', '--'])
    
    # plot support vectors
    if plot_support:
        ax.scatter(model.support_vectors_[:, 0],
                   model.support_vectors_[:, 1],
                   s=300, linewidth=1, facecolors='none');
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

We'll start with the familiar Iris example. The cell below contains the same code we used before, and just puts our feature data into the variable "datapoints", our class labels into the variable "labels", and plots two of our features together to remind us about the dataset:

In [ ]:
iris = sns.load_dataset('iris')

# Remember, we can only directly plot two attributes, e.g. sepal_length vs sepal_width
# Let's choose two to look at first (so we can easily change them later)
feature1 = 'sepal_length'
feature2 = 'petal_length'

species = iris['species'].unique()
colors = {'setosa': 'red', 'versicolor': 'green', 'virginica': 'blue'}

for name in species:
    plt.scatter(iris[iris['species'] == name][feature1], iris[iris['species'] == name][feature2], c=colors[name])

plt.show()

datapoints = iris.drop('species', axis=1).values
labels = np.array(iris['species'])

To begin with, we will make this problem even simpler by turning it into a two-class classification problem: distinguish setosa (red) from versicolor-or-virginica (blue or green). We can do this by modifying our datapoints and labels:

In [ ]:
datapoints = iris[[feature1, feature2]]
datapoints.head()

labels[labels == 'versicolor'] = 'virginica'
labels

for name in species:
    plt.scatter(datapoints[labels == name][feature1], datapoints[labels == name][feature2], c=colors[name])

plt.show()

Let's try training a Support Vector Machine classifier on this. In sklearn, the Support Vector Machine classifier is called "SVC" (Support Vector Classifier). When we do this, we need to supply various hyperparameters:
* A value of C. This determines the tradeoff between correctly classifying points of the training data, versus maximizing the margin around the decision boundary. Higher values of C will favour smaller margins.
* A kernel to use: one of "linear", "rbf" (radial basis function), or "poly" (polynomial)
* **For non-linear kernels** (i.e. any kernel except "linear"): A value of gamma, which affects how far the influence of each training data point extends; low values meaning 'far' and high values meaning 'close'.

Some more detailed descriptions of these (particularly as they affect the RBF kernel) are available in [the documentation](https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html) (don't read that until you've finished this notebook!).

In [ ]:
# The next line creates a classifier, with C set to 0.1
model = SVC(C=0.1, kernel='linear')

# We can fit it to our data in the usual way:
model.fit(datapoints, labels)

# We can (of course) then make predictions for any similar datapoints, e.g. the point (6.1, 5.0):
model.predict([[6.1, 5.0]])

We can use the helper function "plot_svc_decision_function" to plot the decision boundary that our model has created, alongside all of our training data:

In [ ]:
plt.scatter(datapoints.iloc[:, 0], datapoints.iloc[:, 1],  s=50, c=pd.Series(labels).map(colors))
plot_svc_decision_function(model)

In the above chart, the solid line represents the decision boundary, and the dotted lines the margins. By looking at the chart, answer the following:

**Q:**
* Is a linear classifier (e.g. SVM with linear kernel) appropriate for this dataset?
* With C=0.1 (as specified when we created the model), in the model shown are any training datapoints misclassified?

Next, try altering the value of C in the above code, and note what happens to the model. You will need to keep track of what your model looked like in each case; you can either do this by:
* Changing the value of "c" in the "model = SVC(...)" line above, then re-running that cell and the one below, and saving the image created in each model (with a suitable filename) by right-clicking on the image.
* OR: Copying the code from the two cells above into cells below this one, and modifying the value "c" in each copy (so that you end up with one chart for each value of C displayed in this notebook).

In particular, try (at least) the following values:

* 0.1 (the default when you first loaded this notebook)
* 1
* 10
* 100
* 0.01
* 0.005

**Q:** For each value of C, answer the following two questions:
* Are any training datapoints misclassified?
* Do any training datapoints fall within the margin?
* Does the decision boundary drawn look intuitively plausible?

**Q:** Why, intuitively, does increasing the value of C to any number greater than 10 not change the decision boundary or accuracy of the classifier?

We have so far just been building a model using all of our data (i.e. without keeping any test data back to validate on). We can validate our models in the same ways as with any other sklearn classifier, e.g. by using 5-fold cross validation:

In [ ]:
model = SVC(C=0.1, kernel='linear')
scores = cross_val_score(model, datapoints, labels, cv=5)
scores.mean()

**Q:** Using the above cell (you don't need to write any new code - just change C and re-run the cell), find out the cross-validated accuracy for those values of C which you identified above as leading to misclassified datapoints in the training data. Write down the value of C and the mean cross-validation score below.


Grid search
----

It's tedious to have to check values of hyperparameters manually. Instead, we can have sklearn do this for us using a grid search - i.e. repeatedly training and evaluating models for us, each with a different choice of hyperparameters. The cell below makes sklearn perform a grid search with an SVM classifier, using our data, 5-fold cross-validation, and the same five possible values of C we tried in the example above.

Note: the following cell will not display any directly useful output, but will store all of its results in the "gridsearch" variable ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)).

In [ ]:
model = SVC(kernel='linear')
parameters = {
              'C': [0.1, 1, 10, 100, 0.005],
             }
gridsearch = GridSearchCV(estimator=model, param_grid=parameters, cv=5)
gridsearch.fit(datapoints, labels)

The "gridsearch" object contains the results for all the possible choices of hyperparameter we searched for (in this case, just each possible value of C). Often, we just want to know which hyperparameter combination got the best score:

In [ ]:
gridsearch.best_params_

But we can also see the complete list of mean cross-validation scores:

In [ ]:
gridsearch.cv_results_['mean_test_score']

These values correspond to (and are listed in the same order as) the parameters listed under "cv_results_['params']":

In [ ]:
gridsearch.cv_results_['params']

Non-linear SVMs
----

As far as sklearn is concerned, non-linear kernels ('rbf' - radial basis function, and 'poly' - polynomial) function in exactly the same way as linear kernels. The only difference is that they have additional hyperparameters such as gamma.

Although the Iris problem seems suitable for solving with a linear classifier, we can also try the non-linear models here to see what results these would get. First, let's try a RBF with gamma=0.1:

In [ ]:
model = SVC(gamma=0.1, C=0.1, kernel='rbf')
model.fit(datapoints, labels)
plt.scatter(datapoints.iloc[:, 0], datapoints.iloc[:, 1],  s=50, c=pd.Series(labels).map(colors))

plot_svc_decision_function(model);

Again, the solid line is the decision boundary, dotted lines are margins. Try experimenting with different values of C and gamma in the above.

**Q:** Does the model correctly classify the training data with the default values above (i.e. gamma=0.1, C=0.1)?

**Q:** Does it always correctly classify the training data regardless of the values of gamma and C? If not, provide a pair of values for which it does not.

Now that we have more than one hyperparameter (i.e. gamma as well as C), the gridsearch function becomes much more useful. We can use it to find the best set of parameters just as before, but this time we specify possible values of gamma as well as C:

In [ ]:
model = SVC(kernel='rbf')

parameters = {
              'C': [0.1, 1, 100, 1000],
              'gamma': [0.0001, 0.001, 0.005, 0.1, 1, 3, 5]
             }
gridsearch = GridSearchCV(estimator=model, param_grid=parameters, cv=5)
gridsearch.fit(datapoints, labels)

As before, we can find the best-scoring parameters - this time a combination of both:

In [ ]:
gridsearch.best_params_

If we look at the cross-validation results, we'll see there are many more of them this time:

In [ ]:
gridsearch.cv_results_['mean_test_score']

This is because the grid search has tried *all possible combinations* of the parameters we specified. We can see exactly which ones it has tried by using the 'params' entry in 'cv_results_'. Each item in cv_results_['params'] is the score for the corresponding entry in the cv_results_['mean_test_score'] list.

In [ ]:
gridsearch.cv_results_['params']

So for instance, the parameter combination in gridsearch.cv_results_['params'][10] (whatever this is) resulted in a cross-validation score of gridsearch.cv_results_['mean_test_score'][10] (whatever *that* is):

In [ ]:
gridsearch.cv_results_['params'][10]

In [ ]:
gridsearch.cv_results_['mean_test_score'][10]

**Q:** Using this correspondence, which combination of parameters resulted in the *worst* score? You don't need to write any code, just compare the contents of gridsearch.cv_results_['mean_test_score'] and gridsearch.cv_results_['params'] printed out above to answer this.

**Optional/for reference:** Another way to view this data is as a Pandas dataframe; the cv_results_ variable is arranged so that you can easily create a dataframe from it. It contains more data than we need, but is easy to navigate once we work out which columns are important to us, because each row corresponds to one choice of parameters:

In [ ]:
pd.DataFrame(gridsearch.cv_results_)

The polynomial kernel works similarly, but will produce quite different models. Before we move on to harder datasets, let's try this also on the Iris data (feel free to experiment with different values of gamma and C):

In [ ]:
model = SVC(gamma=0.1, C=0.1, kernel='poly')
model.fit(datapoints, labels)

plt.scatter(datapoints.iloc[:, 0], datapoints.iloc[:, 1],  s=50, c=pd.Series(labels).map(colors))
plot_svc_decision_function(model)

**Important note:** The SVM classifiers in sklearn *can* work with more than 2-class problems, and *do* work with more than 2-dimensional data (i.e. >2 features). *However*, these cannot be visualized so easily.

**Optional task:** Evaluate and compare linear, RBF, and polynomial SVMs on the full 3-class Iris dataset in four dimensions (only do this after completing the rest of this notebook).

Solving harder problems
----

The Iris dataset does not need a non-linear classifier to get a good result; some classification problems however cannot be solved well with a linear classifier. We will construct a similar problem to one from the slides (**don't worry about the code in the next cell - we're just using it to create a dataset**):

In [ ]:
datapoints, labels = make_blobs(n_samples=300, centers=3, random_state=753, cluster_std=0.8)
for i in range(0,300):
    if labels[i]==0:
        labels[i]=1
plt.scatter(datapoints[:, 0], datapoints[:, 1], c=labels, s=50, cmap='rainbow');

We can work with this using exactly the same methods as with the Iris data.

**Q:** Are the two classes shown above linearly separable?

**Q:** Are the two classes likely to be perfectly separable with *any* classifier? Why?

Let's start by trying a RBF SVM (we'll make the plot bigger for clarity):

In [ ]:
model = SVC(gamma=0.1, C=0.1, kernel='rbf')
model.fit(datapoints, labels)

# The next line just increases the size on the page of the output chart, so we can better see the details
plt.figure(figsize=(20,15))

plt.scatter(datapoints[:,0], datapoints[:,1],  s=50, c=labels, cmap='rainbow')
plot_svc_decision_function(model);

Now that our decision boundaries are getting more complex, it may be helpful to reuse something we saw in an earlier notebook: creating lots of random datapoints, and asking our trained classifier to predict their classes, so we can see clearly how data is classified by our model. The next cell creates 4000 new random datapoints, and plots these in lighter shades of our two colors, to highlight how our model classifies datapoints (don't worry too much about the details of this):

In [ ]:
rng = np.random.RandomState(0)
Xnew = [-8, -12] + [16, 12] * rng.rand(4000, 2)
ynew = model.predict(Xnew)
plt.figure(figsize=(20,15))
lim = plt.axis()
plt.scatter(Xnew[:, 0], Xnew[:, 1], c=ynew, s=20, cmap='rainbow', alpha=0.1)
plt.scatter(datapoints[:, 0], datapoints[:, 1], c=labels, s=50, cmap='rainbow');
plot_svc_decision_function(model)

The above classifier seems to be doing a reasonable job. Would a polynomial kernel work? Let's try one:

In [ ]:
model = SVC(gamma=1, C=0.1, kernel='poly')
model.fit(datapoints, labels)

rng = np.random.RandomState(0)
Xnew = [-8, -12] + [16, 12] * rng.rand(4000, 2)
ynew = model.predict(Xnew)
plt.figure(figsize=(20,15))
lim = plt.axis()
plt.scatter(Xnew[:, 0], Xnew[:, 1], c=ynew, s=20, cmap='rainbow', alpha=0.1)
plt.scatter(datapoints[:, 0], datapoints[:, 1], c=labels, s=50, cmap='rainbow');
plot_svc_decision_function(model)

**Q:** Experiment manually with changes to the hyperparameters in both the RBF and polynomial examples above.

**Q:** Using a grid search, find optimal combinations of parameters for both RBF and polynomial kernels. **Note:** you can either do this exactly as above, with two grid searches (one for 'poly', one for 'rbf'), *or* you can include 'kernel' as an additional grid search parameter, with the two possible values of 'rbf' and 'poly. **Note:** This may take longer to run! Start with a small number of hyperparameter values.

**Q:** Write a short program using a for loop to build a model with each set of parameter choices made in the grid search, in each case printing out the parameters, the mean k-means validation score, and the decision function (using plot_svc_decision_function). (You may wish to reuse the code from the above cells for this, and if you like include predictions for randomly generated datapoints)